# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import pkg_resources
import joblib

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn import datasets
from sklearn.metrics import confusion_matrix
from azureml.core.conda_dependencies import CondaDependencies
import azureml.core
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig, Environment

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'parkisnon-hyperparam'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [18]:
# Define your environment (you can customize this further)
# myenv = Environment(name="parkinson_env")
# conda_dep = CondaDependencies()
# conda_dep.add_pip_package("scikit-learn")
# myenv.python.conda_dependencies = conda_dep
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Training directory and script
train_dir = "./training"
train_script = "train.py"
# Create a ScriptRunConfig
src = ScriptRunConfig(
    source_directory=train_dir,  # Set your source directory
    script=train_script,  # Set your training script
    compute_target=compute_target,  # Set your compute target
    environment=sklearn_env
)

# Define hyperparameter sampling (modify as needed)
param_sampling = RandomParameterSampling({
    "--n_estimators": choice(50, 100, 150),  # Number of trees
    "--max_depth": choice(10, 20, 30),       # Maximum depth of trees
    "--min_samples_split": uniform(0.1, 0.5)  # Minimum samples to split
    
})

# Define early termination policy (modify as needed)
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Create HyperDriveConfig
hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="r2_score",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=15,
    max_concurrent_runs=5,
    policy=early_termination_policy,
)


In [19]:
#TODO: Submit your experiment
# Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
hyperdrive_run

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3f5a7198-7056-42be-8d8e-ec9443ffc5aa
Web View: https://ml.azure.com/runs/HD_3f5a7198-7056-42be-8d8e-ec9443ffc5aa?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-259743/workspaces/quick-starts-ws-259743&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_3f5a7198-7056-42be-8d8e-ec9443ffc5aa
Web View: https://ml.azure.com/runs/HD_3f5a7198-7056-42be-8d8e-ec9443ffc5aa?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-259743/workspaces/quick-starts-ws-259743&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



Experiment,Id,Type,Status,Details Page,Docs Page
parkisnon-hyperparam,HD_3f5a7198-7056-42be-8d8e-ec9443ffc5aa,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [21]:
hyperdrive_run.get_best_run_by_primary_metric

<bound method HyperDriveRun.get_best_run_by_primary_metric of Run(Experiment: parkisnon-hyperparam,
Id: HD_3f5a7198-7056-42be-8d8e-ec9443ffc5aa,
Type: hyperdrive,
Status: Completed)>

In [22]:
# Get your best run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")

Best run arguments: ['--max_depth', '30', '--min_samples_split', '0.14968159173371887', '--n_estimators', '50']
Best run metrics: {'r2_score': 0.477870806821036}
Best run file names: ['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [25]:
#TODO: Save the best model
# Save the best model
joblib.dump(value='best_run.id', filename="hyperdrive_model.joblib")

['hyperdrive_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

